# Buchbergerアルゴリズムの実装（Poly型を使用）
***

**目的：代数方程式の解を求めたい**
- 1変数：Euclidの互除法により，GCD計算に帰着できる．
- 多変数：GCD計算で発生する割り算の一意性がない.


term orderを導入し，割り算の一般化である単項簡約を定義
- 商と余りの計算→正規化（＝Gのどの要素でも単項簡約できない）

**定義：グレブナー基底**
- $G$がイデアル$I$のグレブナー基底
- $\forall f(x) \in \mathbb{C}[x] ,\quad f(x) \in I \Leftrightarrow f(x) \to^{*}_G 0$
- $\langle ht(I)\rangle = \langle ht(g_1), \dots , ht(g_r)\rangle$
- $\forall f(x) \in I, \quad \exists g_i(x) \in G, ht(g_i) \mid ht(f)$
- $\forall g_i(x), g_j(x) \in G, Spoly(g_i, g_j) \to^{*}_G 0$

**Buchbergerアルゴリズムでの判定条件**
- $G$がイデアル$I$のグレブナー基底 
- $\Leftrightarrow \forall g_i(x), g_j(x) \in G, Spoly(g_i, g_j) \to^{*}_G 0$

## 準備

In [1]:
from sympy import *
from sympy.abc import x, y, z

# monomial = coefficient + term

In [2]:
def to_monomial_list(f):
    f_gens, f_domain = f.gens, f.domain
    string_f = str(f.as_expr())
    string_f = string_f.replace('+', ', ')
    string_f = string_f.replace('-', ', -')
    if string_f[0] == ',':
        string_f = string_f[1:]
    from sympy.parsing.sympy_parser import parse_expr
    monomials = list(parse_expr(string_f))
    monomials_list = []
    for m in monomials:
        m = Poly(m, gens=f_gens, domain=f_domain)
        monomials_list.append(m)
    return monomials_list

In [3]:
def monom2vec(monomial_list, variable_order):
    len_v = len(variable_order)
    #coeffs = f.coeffs()
    vec = []
    coeffs = []
    for m in monomial_list:
        vec.append([degree(m, v) for v in variable_order])
        coeffs.append(m.coeffs()[0])
    vec = Matrix(vec)
    return vec, coeffs

In [4]:
def order_sort(M, monomial_vector, coeffs_list):
    flag = True
    while flag:
        flag = False
        for i in range(monomial_vector.shape[0] - 1):
            v = monomial_vector.row(i) - monomial_vector.row(i+1)
            v = M * v.transpose()
            for j in range(v.shape[0]):
                if v[j] < 0:
                    monomial_vector.row_swap(i, i+1)
                    coeffs_list[i], coeffs_list[i+1] = coeffs_list[i+1], coeffs_list[i]
                    flag = True
                    break
                elif v[j] > 0:
                    break
    return monomial_vector, coeffs_list

## term order (class)

In [5]:
class LexOrder:
    def __init__(self, f, variable_order):
        self.f = f
        monomial_list = to_monomial_list(f)
        monomial_vector, coeffs_list = monom2vec(monomial_list, variable_order)
        len_v = len(variable_order)
        M = eye(len_v)
        monomial_vector, coeffs_list = order_sort(M, monomial_vector, coeffs_list)
        ht_list = monomial_vector.row(0).tolist()[0]
        ht = 1
        for v, e in zip(variable_order, ht_list):
            ht *= v**e
        self.head_term = ht.as_poly(gens=f.gens, domain=f.domain)
        self.head_coefficient = Poly(coeffs_list[0], gens=f.gens, domain=f.domain)
        self.head_monomial = coeffs_list[0]*ht.as_poly(gens=f.gens, domain=f.domain)
    def poly(self):
        return self.f
    def head_monomial(self):
        return self.head_monomial
    def head_coefficient(self):
        return self.head_coefficient
    def head_term(self):
        return self.head_term

In [6]:
class GLexOrder:
    def __init__(self, f, variable_order):
        self.f = f
        monomial_list = to_monomial_list(f)
        monomial_vector, coeffs_list = monom2vec(monomial_list, variable_order)
        len_v = len(variable_order)
        M = eye(len_v)
        M = M.row_insert(0, Matrix([[1 for _ in range(len_v)]]))
        monomial_vector, coeffs_list = order_sort(M, monomial_vector, coeffs_list)
        ht_list = monomial_vector.row(0).tolist()[0]
        ht = 1
        for v, e in zip(variable_order, ht_list):
            ht *= v**e
        self.head_term = ht.as_poly(gens=f.gens, domain=f.domain)
        self.head_coefficient = Poly(coeffs_list[0], gens=f.gens, domain=f.domain)
        self.head_monomial = coeffs_list[0]*ht.as_poly(gens=f.gens, domain=f.domain)
    def poly(self):
        return self.f
    def head_monomial(self):
        return self.head_monomial
    def head_coefficient(self):
        return self.head_coefficient
    def head_term(self):
        return self.head_term

In [7]:
class GrevLexOrder:
    def __init__(self, f, variable_order):
        self.f = f
        monomial_list = to_monomial_list(f)
        monomial_vector, coeffs_list = monom2vec(monomial_list, variable_order)
        len_v = len(variable_order)
        M = []
        for i in range(len_v):
            M.append([0 for _ in range(len_v)])
            M[i][len_v-i-1] = -1
        M = Matrix(M)
        M = M.row_insert(0, Matrix([[1 for _ in range(len_v)]]))
        monomial_vector, coeffs_list = order_sort(M, monomial_vector, coeffs_list)
        ht_list = monomial_vector.row(0).tolist()[0]
        ht = 1
        for v, e in zip(variable_order, ht_list):
            ht *= v**e
        self.head_term = ht.as_poly(gens=f.gens, domain=f.domain)
        self.head_coefficient = Poly(coeffs_list[0], gens=f.gens, domain=f.domain)
        self.head_monomial = coeffs_list[0]*ht.as_poly(gens=f.gens, domain=f.domain)
    def poly(self):
        return self.f
    def head_monomial(self):
        return self.head_monomial
    def head_coefficient(self):
        return self.head_coefficient
    def head_term(self):
        return self.head_term

## 正規化（normalization）

In [47]:
def normalization(f, G, variable_order, term_order=LexOrder):
    from sympy.solvers.diophantine import divisible
    break_flag = False
    all_through_flag = True
    while True:
        f_list = to_monomial_list(f)
        for g in G:
            _g = term_order(g, variable_order)
            ht_g = _g.head_term
            for monomial_f in f_list:
                if divisible(monomial_f, ht_g):
                    h = f - quo(monomial_f, _g.head_monomial)*_g.poly()
                    #print(f.as_expr(), h)
                    print('f : ', f)
                    print('g : ', _g.poly())
                    print('h : ', h)
                    print('-------------------')
                    f = h
                    if f == 0:
                        return f
                    break_flag = True
                    all_through_flag = False
                    break
            if break_flag:
                break_flag = False
                break
        if all_through_flag:
            return f
        all_through_flag = True

In [48]:
h = x**4 + x**3*z**2 - x**3 + x**2*y**2*z - x**2*z**2 - x*y**2*z - x*y + x + 2*y**4 - 2*y**3 + 3*y - 3
h = h.as_poly(x, y, z, domain=ZZ)
#display(h)

f = x**3 + x*y**2*z + x**2*z**2
g = x - 2*y**3 - 3
f, g = f.as_poly(x, y, z), g.as_poly(x, y, z)
normalization(h, [f, g], [x, y, z], LexOrder)

f :  Poly(x**4 + x**3*z**2 - x**3 + x**2*y**2*z - x**2*z**2 - x*y**2*z - x*y + x + 2*y**4 - 2*y**3 + 3*y - 3, x, y, z, domain='ZZ')
g :  Poly(x**3 + x**2*z**2 + x*y**2*z, x, y, z, domain='ZZ')
h :  Poly(-x**3 - x**2*z**2 - x*y**2*z - x*y + x + 2*y**4 - 2*y**3 + 3*y - 3, x, y, z, domain='ZZ')
-------------------
f :  Poly(-x**3 - x**2*z**2 - x*y**2*z - x*y + x + 2*y**4 - 2*y**3 + 3*y - 3, x, y, z, domain='ZZ')
g :  Poly(x**3 + x**2*z**2 + x*y**2*z, x, y, z, domain='ZZ')
h :  Poly(-x*y + x + 2*y**4 - 2*y**3 + 3*y - 3, x, y, z, domain='ZZ')
-------------------
f :  Poly(-x*y + x + 2*y**4 - 2*y**3 + 3*y - 3, x, y, z, domain='ZZ')
g :  Poly(x - 2*y**3 - 3, x, y, z, domain='ZZ')
h :  Poly(x - 2*y**3 - 3, x, y, z, domain='ZZ')
-------------------
f :  Poly(x - 2*y**3 - 3, x, y, z, domain='ZZ')
g :  Poly(x - 2*y**3 - 3, x, y, z, domain='ZZ')
h :  Poly(0, x, y, z, domain='ZZ')
-------------------


Poly(0, x, y, z, domain='ZZ')

In [49]:
if __name__ == '__main__':
    s = Poly(x + 2*y**2*z - 5*z, x, y, z, domain='QQ')
    g1 = Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ')
    g2 = Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ')
    g3 = Poly(x*z - 1, x, y, z, domain='QQ')
    g4 = Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ')
    g5 = Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
    G = [g1, g2, g3, g4, g5]
    normalization(s, G, [x, y, z], LexOrder)

f :  Poly(x + 2*y**2*z - 5*z, x, y, z, domain='QQ')
g :  Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ')
h :  Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
-------------------
f :  Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
g :  Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
h :  Poly(0, x, y, z, domain='QQ')
-------------------


## S polynomial の計算

$$Spoly(f, g) = \frac{lcm(ht(f), ht(g))}{hm(f)}f(x) - \frac{lcm(ht(f), ht(g))}{hm(g)}f(g)$$

In [50]:
def S_polynomial(f, g, variable_order, term_order=LexOrder):
    f, g = term_order(f, variable_order), term_order(g, variable_order)
    LCM = lcm(f.head_term, g.head_term)
    Spoly = quo(LCM, f.head_monomial)*f.poly() - quo(LCM, g.head_monomial)*g.poly()
    return Spoly

## Buchbergerアルゴリズム本体

In [58]:
def _BuchbergerAlgorithm(F, variable_order, term_order=LexOrder):
    import itertools
    G = F
    P = list(itertools.combinations(F, 2))
    while len(P) != 0:
        f, g = term_order(P[0][0], variable_order), term_order(P[0][1], variable_order)
        P.remove((P[0][0], P[0][1]))
        print('======================')
        print('P : ', f.poly(), g.poly())
        print('Spoly : ', S_polynomial(f.poly(), g.poly(), variable_order, term_order))
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            print('in')
            for h in G:
                P.append((h, r))
            G.append(r)
    return G

## 計算例

In [64]:
%%time
f1 = Poly(x**2 + y**2 + z**2 - 1, [x, y, z], domain=QQ)
f2 = Poly(x*y*z - 1, [x, y, z], domain=QQ)
term_order = GrevLexOrder
F = [f1, f2]
_BuchbergerAlgorithm(F, [x, y, z], term_order)

P :  Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ') Poly(x*y*z - 1, x, y, z, domain='QQ')
Spoly :  Poly(x + y**3*z + y*z**3 - y*z, x, y, z, domain='QQ')
in
P :  Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ') Poly(x + y**3*z + y*z**3 - y*z, x, y, z, domain='QQ')
Spoly :  Poly(-x**3 - x**2*y*z**3 + x**2*y*z + y**5*z + y**3*z**3 - y**3*z, x, y, z, domain='QQ')
f :  Poly(-x**3 - x**2*y*z**3 + x**2*y*z + y**5*z + y**3*z**3 - y**3*z, x, y, z, domain='QQ')
g :  Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ')
h :  Poly(-x**2*y*z**3 + x**2*y*z + x*y**2 + x*z**2 - x + y**5*z + y**3*z**3 - y**3*z, x, y, z, domain='QQ')
-------------------
f :  Poly(-x**2*y*z**3 + x**2*y*z + x*y**2 + x*z**2 - x + y**5*z + y**3*z**3 - y**3*z, x, y, z, domain='QQ')
g :  Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ')
h :  Poly(x**2*y*z + x*y**2 + x*z**2 - x + y**5*z + 2*y**3*z**3 - y**3*z + y*z**5 - y*z**3, x, y, z, domain='QQ')
-------------------
f :  Poly(x**2*y*z + x*y**2 + x*z**2 - x + y**5*z + 2*

[Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ'),
 Poly(x*y*z - 1, x, y, z, domain='QQ'),
 Poly(x + y**3*z + y*z**3 - y*z, x, y, z, domain='QQ')]

\begin{align}
f(x) &= x^2 + y^2 + z^2 - 4, \\
g(x) &= x^2 +2y^2 - 5, \\
h(x) &= xz - 1
\end{align}

In [57]:
f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
h = Poly(x*z - 1, (x, y, z), domain=QQ)
F = [f, g, h]
term_order = GrevLexOrder

In [30]:
%%time
_BuchbergerAlgorithm(F, [x, y, z], term_order)

-y**2 + z**2 + 1 Poly(-z, x, y, z, domain='QQ') Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
-y**2 + z**2 + 1 Poly(-2*z, x, y, z, domain='QQ') Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')
x + 2*z**3 - 3*z Poly(1, x, y, z, domain='QQ') Poly(0, x, y, z, domain='QQ')
x**2 + y**2 + z**2 - 4 Poly(z**2, x, y, z, domain='QQ') Poly(x**2 + y**4 - 4*y**2 - z**4 + 4*z**2, x, y, z, domain='QQ')
x**2 + y**2 + z**2 - 4 Poly(1, x, y, z, domain='QQ') Poly(y**4 - 5*y**2 - z**4 + 3*z**2 + 4, x, y, z, domain='QQ')
-y**2 + z**2 + 1 Poly(-y**2, x, y, z, domain='QQ') Poly(y**2*z**2 - 4*y**2 - z**4 + 3*z**2 + 4, x, y, z, domain='QQ')
-y**2 + z**2 + 1 Poly(-z**2, x, y, z, domain='QQ') Poly(-4*y**2 + 4*z**2 + 4, x, y, z, domain='QQ')
-y**2 + z**2 + 1 Poly(4, x, y, z, domain='QQ') Poly(0, x, y, z, domain='QQ')
x**2 + y**2 + z**2 - 4 Poly(z**2, x, y, z, domain='QQ') Poly(x**2 + 2*y**4 - y**2*z**2 - 5*y**2 - z**4 + 4*z**2, x, y, z, domain='QQ')
x**2 + y**2 + z**2 - 4 Poly(1, x, y, z, domain='QQ') Poly(2*y**4 - y*

[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

\begin{align}
f(x) &= x^2 + y^2 + z^2 - 1, \\
g(x) &= xz^2 + y^2z - 1
\end{align}

In [14]:
f = Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain=QQ)
g = Poly(x*z**2 + y**2*z - 1, x, y, z, domain=QQ)
F = [f, g]
term_order = GLexOrder

In [15]:
%%time
_BuchbergerAlgorithm(F, [x, y, z], term_order)

CPU times: user 351 ms, sys: 2.06 ms, total: 353 ms
Wall time: 352 ms


[Poly(x**2 + y**2 + z**2 - 1, x, y, z, domain='QQ'),
 Poly(x*z**2 + y**2*z - 1, x, y, z, domain='QQ'),
 Poly(-x*y**2*z + x + y**2*z**2 + z**4 - z**2, x, y, z, domain='QQ'),
 Poly(x*z + y**4*z + y**2*z**3 - y**2 + z**5 - z**3, x, y, z, domain='QQ')]

---
---

# Buchbergerアルゴリズムの改良（criteria）
***

## Buchbergerアルゴリズムの書き換え

In [16]:
def BuchbergerAlgorithm(F, variable_order, term_order=LexOrder):
    original_F = F
    G = [F[0]]
    P = []
    F = F[1:]
    m = 0
    while len(F) != 0:
        G.append(F[0])
        F = F[1:]
        P = Update_normal(P, m)
        m += 1
    while len(P) != 0:
        i, j = P[0][0], P[0][1]
        f, g = term_order(G[i], variable_order), term_order(G[j], variable_order)
        P = P[1:]
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            G.append(r)
            P = Update_normal(P, m)
            m += 1
    return G

In [17]:
def Update_normal(P, m):
    for i in range(m+1):
        P.append((i, m+1))
    return P

originalでは，$Spoly \to 0$にならなかった$r(x)$で全てのペアを追加していたが，
以下の条件より，$Spoly \to 0$を満たさないペアがあらかじめわかる．
- $\forall f(x), g(x) \in \mathbb{C}[x], \quad \gcd(ht(f), ht(g))=1, \quad Spoly(f, g) \to^{*}_{(f, g)} 0$

この条件に基づいて，ペアの更新条件を次のようにできる．
- $\Leftrightarrow lcm(ht(f), ht(g)) \neq ht(f), ht(g) $のもの

In [18]:
def BuchbergerAlgorithm_criteria1(F, variable_order, term_order=LexOrder):
    G = [F[0]]
    P = []
    F = F[1:]
    m = 0
    while len(F) != 0:
        G.append(F[0])
        F = F[1:]
        P = Update_criteria1(P, m, G, variable_order, term_order)
        m += 1
    while len(P) != 0:
        i, j = P[0][0], P[0][1]
        f, g = term_order(G[i], variable_order), term_order(G[j], variable_order)
        P = P[1:]
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            G.append(r)
            P = Update_criteria1(P, m, G, variable_order, term_order)
            m += 1
    return G

In [19]:
def Update_criteria1(P, m, G, variable_order, term_order):
    ht_g1 = term_order(G[m+1], variable_order).head_term
    for i in range(m+1):
        ht_g2 = term_order(G[i], variable_order).head_term
        if lcm(ht_g1, ht_g2) != ht_g1 * ht_g2:
            P.append((i, m+1))
    return P

更に，次のような十分条件も考えられる．
- $T_{ij}=lcm(ht(g_i), ht(g_j))$とし，
\begin{align}
F(k; i, j) & \Leftrightarrow k<i, \quad T_{jk} = T_{ij} \\
M(k; i, j) & \Leftrightarrow k<j, \quad (ht(g_k) \mid T_{ij}) \land (T_{jk} \neq T_{ij}) \\
B(k; i, j) & \Leftrightarrow k>j, \quad (ht(g_k) \mid T_{ij}) \land (T_{ik} \neq T_{ij}, T_{jk} \neq T_{ij})
\end{align}
とする．
- このとき，
$$\forall(i, j) \in \{(i, j) : \not\exists k, F \lor M \lor B \}, Spoly(g_i, g_j) \to^*_G 0$$

In [20]:
def BuchbergerAlgorithm_criteria2(F, variable_order, term_order=LexOrder):
    G = [F[0]]
    P = []
    F = F[1:]
    m = 0
    while len(F) != 0:
        G.append(F[0])
        F = F[1:]
        P = Update_criteria2(P, m, G, variable_order, term_order)
        m += 1
    while len(P) != 0:
        i, j = P[0][0], P[0][1]
        f, g = term_order(G[i], variable_order), term_order(G[j], variable_order)
        P = P[1:]
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            G.append(r)
            P = Update_criteria2(P, m, G, variable_order, term_order)
            m += 1
    return G

In [21]:
def Update_criteria2(P, m, G, variable_order, term_order=LexOrder):
    from sympy.solvers.diophantine import divisible
    ht_g_mp1 = term_order(G[m+1], variable_order).head_term
    # B成立を除去
    for i, j in P:
        ht_g_i = term_order(G[i], variable_order).head_term
        ht_g_j = term_order(G[j], variable_order).head_term
        if divisible(lcm(ht_g_i, ht_g_j), ht_g_mp1):
            if lcm(ht_g_i, ht_g_j) != lcm(ht_g_i, ht_g_mp1):
                if lcm(ht_g_i, ht_g_j) != lcm(ht_g_j, ht_g_mp1):
                    P.remove((i, j))
    for i in range(m+1):
        continue_flag = False
        ht_g_i = term_order(G[i], variable_order).head_term
        # F成立を確認
        for k in range(i):
            ht_g_k = term_order(G[k], variable_order).head_term
            if lcm(ht_g_k, ht_g_mp1) == lcm(ht_g_i, ht_g_mp1):
                continue_flag = True
                break
        if continue_flag:
            continue
        # M成立を確認
        for k in range(m+1):
            if k == i:
                # k not= "i" ?
                continue
            ht_g_k = term_order(G[k], variable_order).head_term
            if divisible(lcm(ht_g_i, ht_g_mp1), ht_g_k):
                if lcm(ht_g_mp1, ht_g_k) != lcm(ht_g_i, ht_g_mp1):
                    continue_flag = True
                    break
        if continue_flag:
            continue
        # Criteria1 check
        if lcm(ht_g_i, ht_g_mp1) != ht_g_i * ht_g_mp1:
            P.append((i, m+1))
    return P

## 計算例

\begin{align}
f(x) &= x^2 + y^2 + z^2 - 4, \\
g(x) &= x^2 +2y^2 - 5, \\
h(x) &= xz - 1
\end{align}

In [22]:
f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
h = Poly(x*z - 1, (x, y, z), domain=QQ)
F = [f, g, h]
term_order = LexOrder

In [23]:
%%time
BuchbergerAlgorithm(F, [x, y, z], term_order)

[(0, 1), (0, 2), (1, 2)]
CPU times: user 642 ms, sys: 4.94 ms, total: 647 ms
Wall time: 646 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ'),
 Poly(-2*z**4 + 3*z**2 - 1, x, y, z, domain='QQ')]

In [24]:
%%time
BuchbergerAlgorithm_criteria1(F, [x, y, z], term_order)

CPU times: user 228 ms, sys: 1.96 ms, total: 230 ms
Wall time: 229 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ'),
 Poly(-2*z**4 + 3*z**2 - 1, x, y, z, domain='QQ')]

In [25]:
%%time
BuchbergerAlgorithm_criteria2(F, [x, y, z], term_order)

CPU times: user 285 ms, sys: 2.64 ms, total: 288 ms
Wall time: 286 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ'),
 Poly(-2*z**4 + 3*z**2 - 1, x, y, z, domain='QQ')]

---
---

# Buchbergerアルゴリズムの改良（suger strategy）
***

今までは，ペアの取り出しはランダム（実装では頭から）だったが，ペア数をできるだけ増やさないようにペア選択の順番を工夫する．

**normal strategy (by Buchberger)**
- 項順序に関して$T_{ij}$の少ない順に選択する戦略（$T_{ij}=lcm(ht(g_i), ht(g_j))$）

**suger strategy (by CoCoA開発者)**
- S-polynomialに対して，以下で定義されるsuger s の小さい順に選択.　sugerが等しい場合には，normal strategyに基づいて選択する．
    - 入力多項式$f$に対し，$$ s_j = tdeg(f)$$
    - tが単項式のとき，$$ s_{tf} = s_{f} + tdeg(t)$$
    - 多項式の加法に関して，$$s_{f+g} = \max(s_f, s_g)$$

In [26]:
def BuchbergerAlgorithm_normalStrategy(F, variable_order, term_order=LexOrder):
    G = [F[0]]
    P = []
    F = F[1:]
    m = 0
    while len(F) != 0:
        G.append(F[0])
        F = F[1:]
        P = Update_criteria2_normalStrategy(P, m, G, variable_order, term_order)
        m += 1
    while len(P) != 0:
        i, j = P[0][0][0], P[0][0][1]
        f, g = term_order(G[i], variable_order), term_order(G[j], variable_order)
        P = P[1:]
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            G.append(r)
            P = Update_criteria2_normalStrategy(P, m, G, variable_order, term_order)
            m += 1
    return G

## critria 2 の改良（for normal strategy）

In [27]:
def Update_criteria2_normalStrategy(P, m, G, variable_order, term_order=LexOrder):
    from sympy.solvers.diophantine import divisible
    if term_order == LexOrder:
        order_key = 'lex'
    elif term_order == GLexOrder:
        order_key = 'grlex'
    elif term_order == GrevLexOrder:
        order_key = 'grevlex'
    ht_g_mp1 = term_order(G[m+1], variable_order).head_term
    # B成立を除去
    for (i, j), LCM_ij in P:
        ht_g_i = term_order(G[i], variable_order).head_term
        ht_g_j = term_order(G[j], variable_order).head_term
        #LCM_ij = lcm(ht_g_i, ht_g_j)
        if divisible(LCM_ij, ht_g_mp1):
            if LCM_ij != lcm(ht_g_i, ht_g_mp1):
                if LCM_ij != lcm(ht_g_j, ht_g_mp1):
                    P.remove(((i, j), LCM_ij))
    for i in range(m+1):
        continue_flag = False
        ht_g_i = term_order(G[i], variable_order).head_term
        LCM_i_mp1 = lcm(ht_g_i, ht_g_mp1)
        # F成立を確認
        for k in range(i):
            ht_g_k = term_order(G[k], variable_order).head_term
            if lcm(ht_g_k, ht_g_mp1) == LCM_i_mp1:
                continue_flag = True
                break
        if continue_flag:
            continue
        # M成立を確認
        for k in range(m+1):
            if k == i:
                # k not= "i" ?
                continue
            ht_g_k = term_order(G[k], variable_order).head_term
            if divisible(LCM_i_mp1, ht_g_k):
                if lcm(ht_g_mp1, ht_g_k) != LCM_i_mp1:
                    continue_flag = True
                    break
        if continue_flag:
            continue
        # Criteria1 check
        if LCM_i_mp1 != ht_g_i * ht_g_mp1:
            if len(P) == 0:
                P.append(((i, m+1), LCM_i_mp1))
                continue
            for k in range(len(P)):
                larger_one = (LCM_i_mp1 + P[k][1]).as_expr().as_ordered_terms(order=order_key)[0]
                LCM_is_large = larger_one == LCM_i_mp1.as_expr()
                if LCM_is_large:
                    continue
                else:
                    P.insert(k, ((i, m+1), LCM_i_mp1))
                    break
            else:
                P.append(((i, m+1), LCM_i_mp1))
    return P

In [28]:
f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
h = Poly(x*z - 1, (x, y, z), domain=QQ)
F = [f, g, h]
term_order = GLexOrder

In [29]:
%%time
BuchbergerAlgorithm(F, [x, y, z], term_order)

[(0, 1), (0, 2), (1, 2)]
CPU times: user 503 ms, sys: 3.52 ms, total: 507 ms
Wall time: 505 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

In [30]:
%%time
BuchbergerAlgorithm_normalStrategy(F, [x, y, z], term_order)

CPU times: user 176 ms, sys: 2.06 ms, total: 178 ms
Wall time: 177 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

## suger strategy

**[再掲]　suger strategy (by CoCoA開発者)**
- S-polynomialに対して，以下で定義されるsuger $s$ の小さい順に選択.　sugerが等しい場合には，normal strategyに基づいて選択する．
    - 入力多項式$f$に対し，$$ s_j = tdeg(f)$$
    - tが単項式のとき，$$ s_{tf} = s_{f} + tdeg(t)$$
    - 多項式の加法に関して，$$s_{f+g} = \max(s_f, s_g)$$

これに伴い，正規化とS-polynomialの計算を定義し直す．

In [31]:
def normalization_withSugar(f_and_sugar, G, variable_order, term_order=LexOrder):
    from sympy.solvers.diophantine import divisible
    f, f_sugar = f_and_sugar[0], f_and_sugar[1]
    break_flag = False
    all_through_flag = True
    while True:
        f_list = to_monomial_list(f)
        for g, g_sugar in G:
            _g = term_order(g, variable_order)
            ht_g = _g.head_term
            for monomial_f in f_list:
                if divisible(monomial_f, ht_g):
                    monomialF_over_hmG = quo(monomial_f, _g.head_monomial)
                    h = f - monomialF_over_hmG * g
                    h_sugar = max(f_sugar, g_sugar + total_degree(monomialF_over_hmG))
                    f, f_sugar = h, h_sugar
                    if f == 0:
                        return f, f_sugar
                    break_flag = True
                    all_through_flag = False
                    break
            if break_flag:
                break_flag = False
                break
        if all_through_flag:
            return f, f_sugar
        all_through_flag = True

In [32]:
def S_polynomial_withSugar(f_and_sugar, g_and_sugar, variable_order, term_order=LexOrder):
    f, f_sugar = term_order(f_and_sugar[0], variable_order), f_and_sugar[1]
    g, g_sugar = term_order(g_and_sugar[0], variable_order), g_and_sugar[1]
    LCM = lcm(f.head_term, g.head_term)
    LCM_over_htF, LCM_over_htG = quo(LCM, f.head_monomial), quo(LCM, g.head_monomial)
    Spoly = LCM_over_htF * f.poly() - LCM_over_htG * g.poly()
    sugar = max(total_degree(LCM_over_htF) + f_sugar, total_degree(LCM_over_htG) + g_sugar)
    return Spoly, sugar

In [33]:
def BuchbergerAlgorithm_sugarStrategy(F, variable_order, term_order=LexOrder):
    G = [(F[0], total_degree(F[0]))]
    P = []
    F = F[1:]
    m = 0
    while len(F) != 0:
        f = F[0]
        G.append((f, total_degree(f)))
        F = F[1:]
        P = Update_criteria2_sugarStrategy(P, m, G, variable_order, term_order)
        #print(P, '\n-----------')
        m += 1
    while len(P) != 0:
        i, j = P[0][0][0], P[0][0][1]
        f, f_sugar = term_order(G[i][0], variable_order), G[i][1]
        g, g_sugar = term_order(G[j][0], variable_order), G[j][1]
        P = P[1:]
        Spoly, SpolySugar = S_polynomial_withSugar([f.poly(), f_sugar], [g.poly(), g_sugar], variable_order, term_order)
        r, r_sugar = normalization_withSugar([Spoly, SpolySugar], G, variable_order, term_order)
        if r != 0:
            G.append((r, r_sugar))
            P = Update_criteria2_sugarStrategy(P, m, G, variable_order, term_order)
            #print(P, '\n-----------')
            m += 1
    GB = [G[i][0] for i in range(len(G))]
    return GB

In [34]:
def Update_criteria2_sugarStrategy(P, m, G, variable_order, term_order=LexOrder):
    from sympy.solvers.diophantine import divisible
    if term_order == LexOrder:
        order_key = 'lex'
    elif term_order == GLexOrder:
        order_key = 'grlex'
    elif term_order == GrevLexOrder:
        order_key = 'grevlex'
    g_mp1, g_mp1_sugar = G[m+1][0], G[m+1][1]
    
    ht_g_mp1 = term_order(G[m+1][0], variable_order).head_term
    # B成立を除去
    for (i, j), (_, LCM_ij) in P:
        ht_g_i = term_order(G[i][0], variable_order).head_term
        ht_g_j = term_order(G[j][0], variable_order).head_term
        #LCM_ij = lcm(ht_g_i, ht_g_j)
        if divisible(LCM_ij, ht_g_mp1):
            if LCM_ij != lcm(ht_g_i, ht_g_mp1):
                if LCM_ij != lcm(ht_g_j, ht_g_mp1):
                    P.remove(((i, j), (_, LCM_ij)))
    for i in range(m+1):
        continue_flag = False
        g_i, g_i_sugar = G[i][0], G[i][1]
        ht_g_i = term_order(G[i][0], variable_order).head_term
        LCM_i_mp1 = lcm(ht_g_i, ht_g_mp1)
        # F成立を確認
        for k in range(i):
            ht_g_k = term_order(G[k][0], variable_order).head_term
            if lcm(ht_g_k, ht_g_mp1) == LCM_i_mp1:
                continue_flag = True
                break
        if continue_flag:
            continue
        # M成立を確認
        for k in range(m+1):
            if k == i:
                # k not= "i" ?
                continue
            ht_g_k = term_order(G[k][0], variable_order).head_term
            if divisible(LCM_i_mp1, ht_g_k):
                if lcm(ht_g_mp1, ht_g_k) != LCM_i_mp1:
                    continue_flag = True
                    break
        if continue_flag:
            continue
        # Criteria1 check
        if LCM_i_mp1 != ht_g_i * ht_g_mp1:
            Spoly, SpolySugar = S_polynomial_withSugar([g_i, g_i_sugar], [g_mp1, g_mp1_sugar], variable_order, term_order)
            if len(P) == 0:
                P.append(((i, m+1), (SpolySugar, LCM_i_mp1)))
                continue
            else:
                for k in range(len(P)):
                    comparison_SpolySugar, comparison_ht_LCM = P[k][1][0], P[k][1][1]
                    if SpolySugar < comparison_SpolySugar:
                        P.insert(k, ((i, m+1), (SpolySugar, LCM_i_mp1)))
                    elif SpolySugar == comparison_SpolySugar:
                        larger_one = (LCM_i_mp1 + comparison_ht_LCM).as_expr().as_ordered_terms(order=order_key)[0]
                        LCM_is_large = larger_one == LCM_i_mp1.as_expr()
                        if LCM_is_large:
                            continue
                        else:
                            P.insert(k, ((i, m+1), (SpolySugar, LCM_i_mp1)))
                            break
                if k == len(P) - 1:
                    P.append(((i, m+1), (SpolySugar, LCM_i_mp1)))
    return P

In [35]:
f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
h = Poly(x*z - 1, (x, y, z), domain=QQ)
F = [f, g, h]
variable_order = [x, y, z]
term_order = GLexOrder

In [36]:
%%time
display(GroebnerBasis(F, x, y, z, order='grlex'))

GroebnerBasis([2*z**3 + x - 3*z, x**2 + 2*z**2 - 3, x*z - 1, y**2 - z**2 - 1], x, y, z, domain='ZZ', order='grlex')

CPU times: user 6.07 ms, sys: 1.06 ms, total: 7.12 ms
Wall time: 6.45 ms


In [37]:
%%time
BuchbergerAlgorithm(F, variable_order, term_order)

[(0, 1), (0, 2), (1, 2)]
CPU times: user 534 ms, sys: 4.44 ms, total: 538 ms
Wall time: 537 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

In [38]:
%%time
BuchbergerAlgorithm_criteria2(F, variable_order, term_order)

CPU times: user 192 ms, sys: 2.71 ms, total: 195 ms
Wall time: 194 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

In [39]:
%%time
BuchbergerAlgorithm_sugarStrategy(F, variable_order, term_order)

CPU times: user 198 ms, sys: 2.41 ms, total: 201 ms
Wall time: 199 ms


[Poly(x**2 + y**2 + z**2 - 4, x, y, z, domain='QQ'),
 Poly(x**2 + 2*y**2 - 5, x, y, z, domain='QQ'),
 Poly(x*z - 1, x, y, z, domain='QQ'),
 Poly(-y**2 + z**2 + 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ')]

---
---

# 比較
---

## 汎用のアルゴリズム（Criteria, Selection strategy の指定）

In [40]:
def myGroebnerBasis(F, variable_order, term_order, criteria='none', selection_strategy='append'):
    try:
        if selection_strategy == 'sugar':
            return BuchbergerAlgorithm_withSugar(F, variable_order, term_order, criteria, selection_strategy)
        else:
            return BuchbergerAlgorithm_nonSugar(F, variable_order, term_order, criteria, selection_strategy)
    except ValueError:
        raise('Keyword is wrong. criteria="none", "criteria1" or"criteria2". selection_strategy="append", "normal" or "sugar" ')

In [41]:
def minimizeAndNormalization(G, variable_order, term_order):
    from sympy.solvers.diophantine import divisible
    #G_copy = G.copy()
    for g in G:
        _G = G.copy()
        _G.remove(g)
        g = term_order(g, variable_order)
        for _g in _G:
            _g = term_order(_g, variable_order)
            if divisible(_g.head_term, g.head_term):
                G.remove(_g.poly())
    # reduction
    g_index = -1
    for g in G:
        g_index += 1
        _G = G.copy()
        _G.remove(g)
        g = term_order(g, variable_order)
        g = normalization(g.poly(), _G, variable_order, term_order)
        g_coeffs_list = g.coeffs()
        q_list = []
        for coeffs in g_coeffs_list:
            if coeffs.q != 1:
                q_list.append(coeffs.q)
        if len(q_list) != 0:
            #g *= Rational(1, lcm(q_list))
            g *= lcm(q_list)
        if term_order(g, variable_order).head_coefficient.as_expr() < 0:
            g *= -1
        G[g_index] = g

In [42]:
def BuchbergerAlgorithm_nonSugar(F, variable_order, term_order, criteria, selection_strategy):
    Update = {'none': Update_normal, 'criteria1': Update_criteria1, 'criteria2': Update_criteria2}
    #strategy = {'append': append_strategy, 'normal': normal_strategy, 'suger': sugar_strategy}
    
    G = [F[0]]
    P = []
    F = F[1:]
    m = 0
    Update_function = Update[criteria]
    while len(F) != 0:
        G.append(F[0])
        F = F[1:]
        P = Update_function(P, m, G, variable_order, term_order, selection_strategy)
        m += 1
    while len(P) != 0:
        if selection_strategy == 'append':
            i, j = P[0][0], P[0][1]
        else:
            i, j = P[0][0][0], P[0][0][1]
        #print(i, j)
        f, g = term_order(G[i], variable_order), term_order(G[j], variable_order)
        #print(f.poly(), g.poly())
        P = P[1:]
        r = normalization(S_polynomial(f.poly(), g.poly(), variable_order, term_order), G, variable_order, term_order)
        if r != 0:
            G.append(r)
            P = Update_function(P, m, G, variable_order, term_order, selection_strategy)
            m += 1
    minimizeAndNormalization(G, variable_order, term_order)
    return G

In [43]:
def BuchbergerAlgorithm_withSugar(F, variable_order, term_order, criteria, selection_strategy):
    Update = {'none': Update_normal, 'criteria1': Update_criteria1, 'criteria2': Update_criteria2}
    #strategy = {'append': append_strategy, 'normal': normal_strategy, 'sugar': sugar_strategy}
    
    G = [(F[0], total_degree(F[0]))]
    P = []
    F = F[1:]
    m = 0
    Update_function = Update[criteria]
    while len(F) != 0:
        f = F[0]
        G.append((f, total_degree(f)))
        F = F[1:]
        P = Update_function(P, m, G, variable_order, term_order, selection_strategy)
        #print([P[i][1][0] for i in range(len(P))])
        m += 1
    while len(P) != 0:
        i, j = P[0][0][0], P[0][0][1]
        f, f_sugar = term_order(G[i][0], variable_order), G[i][1]
        g, g_sugar = term_order(G[j][0], variable_order), G[j][1]
        P = P[1:]
        Spoly, SpolySugar = S_polynomial_withSugar([f.poly(), f_sugar], [g.poly(), g_sugar], 
                                                   variable_order, term_order)
        r, r_sugar = normalization_withSugar([Spoly, SpolySugar], G, variable_order, term_order)
        if r != 0:
            G.append((r, r_sugar))
            #P = Update_criteria2_sugarStrategy(P, m, G, variable_order, term_order)
            P = Update_function(P, m, G, variable_order, term_order, selection_strategy)
            #print(P, '\n-----------')
            m += 1
    GB = [G[i][0] for i in range(len(G))]
    minimizeAndNormalization(GB, variable_order, term_order)
    return GB

In [44]:
def Update_normal(P, m, G, variable_order, term_order, selection_strategy):
    strategy = {'append': append_strategy, 'normal': normal_strategy, 'sugar': sugar_strategy}
    strategy_function = strategy[selection_strategy]
    for i in range(m+1):
        P = strategy_function(P, m, G, variable_order, term_order, i)
    #print(P)
    return P

In [45]:
def Update_criteria1(P, m, G, variable_order, term_order, selection_strategy):
    strategy = {'append': append_strategy, 'normal': normal_strategy, 'sugar': sugar_strategy}
    strategy_function = strategy[selection_strategy]
    if selection_strategy == 'sugar':
        g_mp1 = term_order(G[m+1][0], variable_order)
    else:
        g_mp1 = term_order(G[m+1], variable_order)
    ht_g_mp1 = g_mp1.head_term
    for i in range(m+1):
        if selection_strategy == 'sugar':
            g_i = term_order(G[i][0], variable_order)
        else:
            g_i = term_order(G[i], variable_order)
        ht_g_i = g_i.head_term
        if lcm(ht_g_mp1, ht_g_i) != ht_g_mp1 * ht_g_i:
            P = strategy_function(P, m, G, variable_order, term_order, i)
            #P = strategy_function
    return P

In [46]:
def Update_criteria2(P, m, G, variable_order, term_order, selection_strategy):
    strategy = {'append': append_strategy, 'normal': normal_strategy, 'sugar': sugar_strategy}
    strategy_function = strategy[selection_strategy]
    from sympy.solvers.diophantine import divisible
    if selection_strategy == 'sugar':
         ht_g_mp1 = term_order(G[m+1][0], variable_order).head_term
    else:
         ht_g_mp1 = term_order(G[m+1], variable_order).head_term
    # B成立を除去
    P_index = -1
    for i1, i2 in P:
        P_index += 1
        if selection_strategy == 'append':
            i, j = i1, i2
        else:
            i, j = i1[0], i1[1]
        if selection_strategy == 'sugar':
            ht_g_i = term_order(G[i][0], variable_order).head_term
            ht_g_j = term_order(G[j][0], variable_order).head_term
        else:
            ht_g_i = term_order(G[i], variable_order).head_term
            ht_g_j = term_order(G[j], variable_order).head_term
        if divisible(lcm(ht_g_i, ht_g_j), ht_g_mp1):
            if lcm(ht_g_i, ht_g_j) != lcm(ht_g_i, ht_g_mp1):
                if lcm(ht_g_i, ht_g_j) != lcm(ht_g_j, ht_g_mp1):
                    del P[P_index]
    for i in range(m+1):
        continue_flag = False
        if selection_strategy == 'sugar':
            ht_g_i = term_order(G[i][0], variable_order).head_term
        else:
            ht_g_i = term_order(G[i], variable_order).head_term
        # F成立を確認
        for k in range(i):
            if selection_strategy == 'sugar':
                ht_g_k = term_order(G[k][0], variable_order).head_term
            else:
                ht_g_k = term_order(G[k], variable_order).head_term
            if lcm(ht_g_k, ht_g_mp1) == lcm(ht_g_i, ht_g_mp1):
                continue_flag = True
                break
        if continue_flag:
            continue
        # M成立を確認
        for k in range(m+1):
            if k == i:
                # k not= "i" ?
                continue
            if selection_strategy == 'sugar':
                ht_g_k = term_order(G[k][0], variable_order).head_term
            else:
                ht_g_k = term_order(G[k], variable_order).head_term
            if divisible(lcm(ht_g_i, ht_g_mp1), ht_g_k):
                if lcm(ht_g_mp1, ht_g_k) != lcm(ht_g_i, ht_g_mp1):
                    continue_flag = True
                    break
        if continue_flag:
            continue
        # Criteria1 check
        if lcm(ht_g_i, ht_g_mp1) != ht_g_i * ht_g_mp1:
            P = strategy_function(P, m, G, variable_order, term_order, i)
            #P = strategy_function
    return P

In [47]:
def append_strategy(P, m, G, variable_order, term_order, i):
    # 入ってきたPを適切な順番に並び替える.
    P.append((i, m+1))
    return P

In [48]:
def normal_strategy(P, m, G, variable_order, term_order, i):
    # 入ってきたPを適切な順番に並び替える．
    if term_order == LexOrder:
        order_key = 'lex'
    elif term_order == GLexOrder:
        order_key = 'grlex'
    elif term_order == GrevLexOrder:
        order_key = 'grevlex'
    ht_g_i = term_order(G[i], variable_order).head_term
    ht_g_mp1 = term_order(G[m+1], variable_order).head_term
    LCM_i_mp1 = lcm(ht_g_i, ht_g_mp1)
    if len(P) == 0:
        P.append(((i, m+1), LCM_i_mp1))
    for k in range(len(P)):
        larger_one = (LCM_i_mp1 + P[k][1]).as_expr().as_ordered_terms(order=order_key)[0]
        LCM_is_large = larger_one == LCM_i_mp1.as_expr()
        if LCM_is_large:
            continue
        else:
            P.insert(k, ((i, m+1), LCM_i_mp1))
            break
    else:
        P.append(((i, m+1), LCM_i_mp1))
    return P

In [49]:
def sugar_strategy(P, m, G, variable_order, term_order, i):
    # 入ってきたPを適切な順番に並び替える．
    # Criteria1 check
    #if LCM_i_mp1 != ht_g_i * ht_g_mp1:
    if term_order == LexOrder:
        order_key = 'lex'
    elif term_order == GLexOrder:
        order_key = 'grlex'
    elif term_order == GrevLexOrder:
        order_key = 'grevlex'
    g_i, g_i_sugar = term_order(G[i][0], variable_order), G[i][1]
    g_mp1, g_mp1_sugar = term_order(G[m+1][0], variable_order), G[m+1][1]
    Spoly, SpolySugar = S_polynomial_withSugar([g_i.poly(), g_i_sugar], [g_mp1.poly(), g_mp1_sugar],
                                               variable_order, term_order)
    LCM_i_mp1 = lcm(g_i.head_term, g_mp1.head_term)
    if len(P) == 0:
        P.append(((i, m+1), (SpolySugar, LCM_i_mp1)))
    else:
        for k in range(len(P)):
            comparison_SpolySugar, comparison_ht_LCM = P[k][1][0], P[k][1][1]
            if SpolySugar < comparison_SpolySugar:
                P.insert(k, ((i, m+1), (SpolySugar, LCM_i_mp1)))
            elif SpolySugar == comparison_SpolySugar:
                larger_one = (LCM_i_mp1 + comparison_ht_LCM).as_expr().as_ordered_terms(order=order_key)[0]
                LCM_is_large = larger_one == LCM_i_mp1.as_expr()
                if LCM_is_large:
                    continue
                else:
                    P.insert(k, ((i, m+1), (SpolySugar, LCM_i_mp1)))
                    break
        if k == len(P) - 1:
            P.append(((i, m+1), (SpolySugar, LCM_i_mp1)))
    return P

In [50]:
f = Poly(x**2 + y**2 + z**2 - 4, (x, y, z), domain=QQ)
g = Poly(x**2 + 2*y**2 -5, (x, y, z), domain=QQ)
h = Poly(x*z - 1, (x, y, z), domain=QQ)
F = [f, g, h]
variable_order = [x, y, z]
term_order = LexOrder

In [51]:
%%time
myGroebnerBasis(F, variable_order, term_order, criteria='criteria2', selection_strategy='sugar')

CPU times: user 454 ms, sys: 3.94 ms, total: 458 ms
Wall time: 459 ms


[Poly(y**2 - z**2 - 1, x, y, z, domain='QQ'),
 Poly(x + 2*z**3 - 3*z, x, y, z, domain='QQ'),
 Poly(2*z**4 - 3*z**2 + 1, x, y, z, domain='QQ')]

In [52]:
GroebnerBasis(F, x, y, z, order='lex')

GroebnerBasis([x + 2*z**3 - 3*z, y**2 - z**2 - 1, 2*z**4 - 3*z**2 + 1], x, y, z, domain='ZZ', order='lex')

## ベンチマーク（cyclic）

In [54]:
def cyclicPoly(n):
    variables = [Symbol('x' + str(i)) for i in range(n)]
    #variables = [Symbol('x'), Symbol('y'), Symbol('z'), Symbol('t'), Symbol('u')]
    F = []
    for k in range(n):
        delta = Integer((k+1)==n)
        sum_f = Poly(0, variables, domain=QQ)
        for i in range(n):
            mul_f = Poly(1, variables, domain=QQ)
            for j in range(i, k+i+1):
                x_index = (j+1)%n - 1
                mul_f *= variables[x_index]
            sum_f += mul_f
            if k == n-1:
                break
        f = sum_f - delta
        F.append(f)
    return F

In [55]:
%%time
cyclicPoly(5)

CPU times: user 16.8 ms, sys: 855 µs, total: 17.6 ms
Wall time: 17.3 ms


[Poly(x0 + x1 + x2 + x3 + x4, x0, x1, x2, x3, x4, domain='QQ'),
 Poly(x0*x1 + x0*x4 + x1*x2 + x2*x3 + x3*x4, x0, x1, x2, x3, x4, domain='QQ'),
 Poly(x0*x1*x2 + x0*x1*x4 + x0*x3*x4 + x1*x2*x3 + x2*x3*x4, x0, x1, x2, x3, x4, domain='QQ'),
 Poly(x0*x1*x2*x3 + x0*x1*x2*x4 + x0*x1*x3*x4 + x0*x2*x3*x4 + x1*x2*x3*x4, x0, x1, x2, x3, x4, domain='QQ'),
 Poly(x0*x1*x2*x3*x4 - 1, x0, x1, x2, x3, x4, domain='QQ')]

In [56]:
"""cyclic_degree = [3, 5]
iternum = 1
criteria_keys = ['none', 'criteria1', 'criteria2']
strategy_keys = ['append', 'normal', 'sugar']
term_order = GrevLexOrder

key_list = []
for c_key in criteria_keys:
    for s_key in strategy_keys:
        key_list.append((c_key, s_key))
print(key_list)"""

"cyclic_degree = [3, 5]\niternum = 1\ncriteria_keys = ['none', 'criteria1', 'criteria2']\nstrategy_keys = ['append', 'normal', 'sugar']\nterm_order = GrevLexOrder\n\nkey_list = []\nfor c_key in criteria_keys:\n    for s_key in strategy_keys:\n        key_list.append((c_key, s_key))\nprint(key_list)"

In [57]:
%%time
'''"""SymPyのGroebnerBasis"""
n = 5
F = cyclicPoly(n)
variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]
term_order = GrevLexOrder
GroebnerBasis(F, variable_order, order='grevlex')'''

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


'"""SymPyのGroebnerBasis"""\nn = 5\nF = cyclicPoly(n)\nvariable_order = [Symbol(\'x\' + str(i)) for i in reversed(range(n))]\nterm_order = GrevLexOrder\nGroebnerBasis(F, variable_order, order=\'grevlex\')'

In [58]:
%%time
'''"""mySimpleGroebner"""
n = 5
F = cyclicPoly(n)
variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]
term_order = GrevLexOrder
_BuchbergerAlgorithm(F, variable_order, term_order)'''

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 3.1 µs


'"""mySimpleGroebner"""\nn = 5\nF = cyclicPoly(n)\nvariable_order = [Symbol(\'x\' + str(i)) for i in reversed(range(n))]\nterm_order = GrevLexOrder\n_BuchbergerAlgorithm(F, variable_order, term_order)'

In [59]:
%%time
"""myGroebnerBasis"""
n = 3
F = cyclicPoly(n)
variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]
term_order = GrevLexOrder
myGroebnerBasis(F, variable_order, term_order, criteria='criteria2', selection_strategy='sugar')

CPU times: user 333 ms, sys: 5.43 ms, total: 338 ms
Wall time: 341 ms


[Poly(x0 + x1 + x2, x0, x1, x2, domain='QQ'),
 Poly(x0**2 + x0*x1 + x1**2, x0, x1, x2, domain='QQ'),
 Poly(x0**3 - 1, x0, x1, x2, domain='QQ')]

In [60]:
%%time
"""myGroebnerBasis"""
n = 4
F = cyclicPoly(n)
variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]
term_order = GrevLexOrder
myGroebnerBasis(F, variable_order, term_order, criteria='criteria2', selection_strategy='sugar')

CPU times: user 4.01 s, sys: 23.7 ms, total: 4.03 s
Wall time: 4.05 s


[Poly(x0 + x1 + x2 + x3, x0, x1, x2, x3, domain='QQ'),
 Poly(x0**2 + 2*x0*x2 + x2**2, x0, x1, x2, x3, domain='QQ'),
 Poly(-x0**3 - x0**2*x2 + x0*x1**2 + x1**2*x2, x0, x1, x2, x3, domain='QQ'),
 Poly(-x0**4 + x0**3*x1 - x0**3*x2 + x0**2*x1**2 + x0**2*x1*x2 - 1, x0, x1, x2, x3, domain='QQ'),
 Poly(x0**5 + x0**4*x2 - x0 - x2, x0, x1, x2, x3, domain='QQ'),
 Poly(x0**3*x1**2 + x0**2*x1**3 - x0 - x1, x0, x1, x2, x3, domain='QQ'),
 Poly(x0**4*x1**2 - 2*x0**2 + x0*x1 - x0*x2 + x1*x2, x0, x1, x2, x3, domain='QQ')]

In [61]:
"""import time
import matplotlib.pyplot as plt

total_time_list = []
for criteria_key, strategy_key in key_list:
    total_time = []
    for n in cyclic_degree:
        F = cyclicPoly(n)
        variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]
        s_time = time.process_time()
        for i in range(iternum):
            GB = myGroebnerBasis(F, variable_order=variable_order, term_order=term_order, 
                            criteria=criteria_key, selection_strategy=strategy_key)
        e_time = time.process_time()
        total_time.append((e_time - s_time)/iternum)
    total_time_list.append(total_time)

fig, ax = plt.subplots()
x_list = cyclic_degree
time_list_index = -1
for criteria_key, strategy_key in key_list:
    time_list_index += 1
    ax.plot(x_list, total_time_list[time_list_index], label=criteria_ley + ' + ' + strategy_key)
    ax.legend()
ax.set_xlabel('cyclic degree')
ax.set_ylabel('time[sec]')
fig.tight_layout()
fig.show()"""

"import time\nimport matplotlib.pyplot as plt\n\ntotal_time_list = []\nfor criteria_key, strategy_key in key_list:\n    total_time = []\n    for n in cyclic_degree:\n        F = cyclicPoly(n)\n        variable_order = [Symbol('x' + str(i)) for i in reversed(range(n))]\n        s_time = time.process_time()\n        for i in range(iternum):\n            GB = myGroebnerBasis(F, variable_order=variable_order, term_order=term_order, \n                            criteria=criteria_key, selection_strategy=strategy_key)\n        e_time = time.process_time()\n        total_time.append((e_time - s_time)/iternum)\n    total_time_list.append(total_time)\n\nfig, ax = plt.subplots()\nx_list = cyclic_degree\ntime_list_index = -1\nfor criteria_key, strategy_key in key_list:\n    time_list_index += 1\n    ax.plot(x_list, total_time_list[time_list_index], label=criteria_ley + ' + ' + strategy_key)\n    ax.legend()\nax.set_xlabel('cyclic degree')\nax.set_ylabel('time[sec]')\nfig.tight_layout()\nfig.show(